In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import numpy as np
import pandas as pd
import re
import nltk
import spacy
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

pd.options.mode.chained_assignment = None

df = pd.read_csv('/kaggle/input/customer-support-on-twitter/twcs/twcs.csv',
                nrows = 500)

In [2]:
df.head(5)

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0


In [3]:
#one hot encoding on the column "Text"

In [8]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
text_encoded = encoder.fit_transform(df[['text']])

text_encoded_df = pd.DataFrame(text_encoded.toarray(), columns=encoder.get_feature_names_out(['text']))

text_encoded_df

,text_$3 burritos and I’m nowhere near a @ChipotleTweets 🤦🏾‍♂️,text_.@AmazonHelp Item has not been delivered but tracking says it was handed to me over an hour ago... 2nd time this has happened. Sort it out https://t.co/42W82GcARk,text_.@VerizonSupport @115725 @115726 &gt;All of VERIZON IS DOWN&lt;\nWhen can we expect a fix ?,text_.@delta this has been my inflight studio experience today. Nothing works except Twitter #toobadforyouandme https://t.co/VLo4ufL73y,"text_1 hour+ inside an old Altima that smells and has ripped stain seats, with an inexperienced miami driver @115873 standards down now 4 applying!",text_@108162 @115761 @ATVIAssist tried that there was people so i came back,"text_@115712 Can you please send us a private message, so that I can gain further details about your account?",text_@115712 I understand. I would like to assist you. We would need to get you into a private secured link to further assist.,text_@115712 Please send us a Private Message so that we can further assist you. Just click ‘Message’ at the top of your profile.,"text_@115713 H there! We'd definitely like to work with you on this, how long have you been experiencing this issue? -AA",...,text_Why are my I’s question marks ?,"text_Worth the squeeze: when you’ve got us helping you along the way, owning your own #smallbiz can be quite fruitful! https://t.co/AgBM1TRAcD","text_Yo @Ask_Spectrum, your customer service reps are super nice— but imma start trippin if y’all don’t get my service going!",text_actually that's a broken link you sent me and incorrect information https://t.co/V4yfrHR8VI,text_amazonのfireTVstickが見れない😢,text_amazonプライムビデオ、再生エラーが多いです,text_hey @ChipotleTweets wanna come to Mammoth. I'll at least eat there once a week i promise,text_somebody from @VerizonSupport please help meeeeee 😩😩😩😩 I'm having the worst luck with your customer service,text_whats wrong w/ amazon leavin shit outside like ups...🤔,"text_‘Ere @marksandspencer, never mind avocado 🥑 - How’s about ‘ave a word?\nIs it really necessary? \n\nThe pair of ‘em were suffocating. https://t.co/ff2bmOVido"
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
496,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
498,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#BOW on the column "Text"

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
bow_matrix = vectorizer.fit_transform(df['text'])


bow_df = pd.DataFrame(bow_matrix.toarray(), columns=vectorizer.get_feature_names_out())
bow_df

,01,0326600,0g98rtnxpk,10,10307,108162,11,111,115712,115713,...,保証期間も過ぎてるので買い直しになるんでしょうね,再生エラーが多いです,強制停止,恐れ入ります,改善しない場合は,状況を確認しご案内させていただきますのでこちらからカスタマーサービスまでご連絡ください,端末の再起動にて改善する場合がございますので,至らない点も多々あるかとは存じますが,電話で主人が対応していただいてます,電話で対応してもらいましたが改良されませんでした
0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
496,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
497,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
498,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#N-grams on the column "Text"

In [15]:

vectorizer = CountVectorizer(ngram_range=(2, 4))

ngram_matrix = vectorizer.fit_transform(df['text'])

ngram_df = pd.DataFrame(ngram_matrix.toarray(), columns=vectorizer.get_feature_names_out())

ngram_df

,01 taipei,01 taipei manila,01 taipei manila __email__,0326600 please,0326600 please check,0326600 please check on,10 31,10 31 3pm,10 31 3pm close,10 dans,...,改善しない場合は 状況を確認しご案内させていただきますのでこちらからカスタマーサービスまでご連絡ください https co,状況を確認しご案内させていただきますのでこちらからカスタマーサービスまでご連絡ください https,状況を確認しご案内させていただきますのでこちらからカスタマーサービスまでご連絡ください https co,状況を確認しご案内させていただきますのでこちらからカスタマーサービスまでご連絡ください https co ntnax2qh2u,端末の再起動にて改善する場合がございますので お試しください,端末の再起動にて改善する場合がございますので お試しください 改善しない場合は,端末の再起動にて改善する場合がございますので お試しください 改善しない場合は 状況を確認しご案内させていただきますのでこちらからカスタマーサービスまでご連絡ください,至らない点も多々あるかとは存じますが 今後ともどうぞよろしくお願いします,至らない点も多々あるかとは存じますが 今後ともどうぞよろしくお願いします et,電話で対応してもらいましたが改良されませんでした 保証期間も過ぎてるので買い直しになるんでしょうね
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
496,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
497,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
498,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#tf-idf on the column "Text"

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
bow_matrix = vectorizer.fit_transform(df['text'])


tf_idf_df = pd.DataFrame(bow_matrix.toarray(), columns=vectorizer.get_feature_names_out())
tf_idf_df

,01,0326600,0g98rtnxpk,10,10307,108162,11,111,115712,115713,...,保証期間も過ぎてるので買い直しになるんでしょうね,再生エラーが多いです,強制停止,恐れ入ります,改善しない場合は,状況を確認しご案内させていただきますのでこちらからカスタマーサービスまでご連絡ください,端末の再起動にて改善する場合がございますので,至らない点も多々あるかとは存じますが,電話で主人が対応していただいてます,電話で対応してもらいましたが改良されませんでした
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.262213,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.276078,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
498,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
#5 advance and disadvantage of one hot encoding, BOW, N-grams, tf-idf



**> #5 advance and disadvantage of one hot encoding, BOW, N-grams, tf-idf**

> One-Hot Encoding:

**Advantages:**

Simple Representation: It converts categorical variables into a format that can be provided to ML algorithms without any issues.

Interpretability: Each category is represented by a binary vector, making it easy to interpret and analyze.

Works with Non-Ordinal Data: Suitable for nominal data where there is no inherent order or ranking among categories.

No Assumptions: One-hot encoding does not impose any assumptions about the data distribution or relationships between categories.

Compatible with Most Models: It can be used with various machine learning algorithms, including linear models, decision trees, and neural networks.

**Disadvantages:**

High Dimensionality: One-hot encoding increases the dimensionality of the dataset significantly, especially when dealing with categorical features with many unique values.

Sparsity: It results in sparse matrices, where most entries are zero, leading to computational inefficiency and memory overhead.

Loss of Information: Each category is treated independently, ignoring any relationships or similarities between them.

Curse of Dimensionality: High-dimensional data may lead to overfitting, especially with models like k-nearest neighbors (k-NN).

Not Suitable for Ordinal Data: It does not capture any ordinal relationships between categories, treating them as equally distant from each other.




> Bag-of-Words (BOW):

**Advantages:**

Simple and Intuitive: BOW representation is easy to understand and implement, making it suitable for text classification and sentiment analysis tasks.
Efficient: It is computationally efficient and scales well to large datasets and vocabularies.

No Semantics Required: BOW treats documents as unordered collections of words, making it language-agnostic and not reliant on syntactic or semantic analysis.

Versatility: BOW can be combined with various machine learning algorithms, including naive Bayes, logistic regression, and support vector machines (SVM).

Captures Word Frequency: It preserves information about word frequency, which can be informative for some tasks like spam detection.

**Disadvantages:**

Lack of Context: BOW ignores word order and context, leading to loss of sequential information in text data.

Sparse Representation: Like one-hot encoding, BOW can result in high-dimensional, sparse matrices, which may pose challenges for memory and computation.

No Semantic Understanding: It treats all words as independent features, disregarding their semantic relationships or meanings.

Vocabulary Size: The size of the vocabulary can grow rapidly, especially for large and diverse datasets, leading to memory and computational overheads.

Not Suitable for Semantic Tasks: BOW is not well-suited for tasks requiring understanding of word semantics or relationships, such as word embeddings or natural language understanding.

> N-grams:

**Advantages:**

Captures Local Context: N-grams capture local word order and context, providing more contextual information compared to BOW.

Better Performance: N-grams can improve the performance of text classification and sentiment analysis models by incorporating some degree of sequence information.

Flexibility: By adjusting the value of "n," you can control the level of context captured, allowing for flexibility in modeling.

Handles Stopwords: Unlike BOW, N-grams can capture the co-occurrence of stopwords with meaningful words, potentially providing additional context.

Compatible with Many Models: N-grams can be used with a wide range of models, including logistic regression, decision trees, and neural networks.


**Disadvantages:**

Data Sparsity: Higher-order N-grams may suffer from data sparsity issues, especially for rare or unseen combinations of words.

Increased Dimensionality: Like BOW, higher-order N-grams can lead to high-dimensional feature spaces, which may require more computational resources.

Loss of Semantic Meaning: N-grams may still struggle to capture semantic relationships between words, as they primarily focus on local word order.

Sensitive to Noise: N-grams can capture noisy or irrelevant patterns in the data, leading to overfitting, especially with small datasets.

Limited Contextual Understanding: While N-grams capture local context, they may not adequately represent long-range dependencies or global semantic meaning in text data.

> TF-IDF (Term Frequency-Inverse Document Frequency):

**Advantages:**

Weighting by Importance: TF-IDF weights terms by their importance in documents, helping to emphasize informative words while downplaying common terms.

Reduces Noise: By penalizing frequent terms and rewarding rare terms, TF-IDF can help reduce the impact of noisy or uninformative words.

Handles Large Vocabularies: TF-IDF is effective in handling large vocabularies and diverse text corpora, making it suitable for text mining and information retrieval tasks.

Language Independence: TF-IDF is language-independent and can be applied to texts in any language without modification.

Interpretability: The resulting TF-IDF matrix retains some interpretability, as it represents the importance of terms relative to documents.

**Disadvantages:**

Requires Preprocessing: TF-IDF requires preprocessing steps like tokenization, stopword removal, and normalization, which may introduce additional complexity.

Loss of Context: Like BOW, TF-IDF disregards word order and context, potentially losing valuable sequential information in text data.

Sparse Representations: TF-IDF can result in sparse matrices, especially for large and diverse text corpora, which may pose computational challenges.

Difficulty Capturing Semantics: TF-IDF does not capture semantic relationships between words, limiting its effectiveness for tasks requiring semantic understanding.

Limited to Single Words: TF-IDF typically operates at the level of individual words, missing out on the benefits of capturing multi-word expressions or phrases.